#**IMPORT LIBRARY & DATA**

In [ ]:
import numpy as np                
import pandas as pd               
import matplotlib.pyplot as plt   
import seaborn as sns             
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore')

pd.set_option('float_format', '{:.2f}'.format)  # Show float data in two decimal number

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
raw_df = pd.read_csv('/content/gdrive/MyDrive/Final Project/bank-additional/bank-additional-full.csv', sep=';')
raw_sample_df = pd.read_csv('/content/gdrive/MyDrive/Final Project/bank-additional/bank-additional.csv', sep=';')

In [ ]:
raw_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no


In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

# **CLEAN DATA**

## YES/NO VALUE TO NUMERIC DATA

In [ ]:
df = raw_df.applymap(lambda x: 1 if x == 'yes' else (0 if x == 'no' else x))
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,0,0,0,telephone,may,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
1,57,services,married,high.school,unknown,0,0,telephone,may,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
2,37,services,married,high.school,0,1,0,telephone,may,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
3,40,admin.,married,basic.6y,0,0,0,telephone,may,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
4,56,services,married,high.school,0,0,1,telephone,may,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0


## CREATE AGE GROUP COLUMN

In [ ]:
# Create age group column for training data

conditions = [(df['age'] < 25),
              (df['age'] < 35) & (df['age'] >= 25),
              (df['age'] < 60) & (df['age'] >= 35),
              (df['age'] >= 60)]

values = [1,2,3,4]

df['age_group'] = np.select(conditions, values)

In [ ]:
# Arrange columns order
df = df[[df.columns[0]] + [df.columns[-1]] + list(df.columns[1:-1])]

In [ ]:
df.age_group = df.age_group.astype(object)

df.head()

,age,age_group,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,housemaid,married,2,0,0.00,0.00,telephone,may,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
1,57,3,services,married,5,unknown,0.00,0.00,telephone,may,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
2,37,3,services,married,5,0,1.00,0.00,telephone,may,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
3,40,3,admin.,married,3,0,0.00,0.00,telephone,may,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
4,56,3,services,married,5,0,0.00,1.00,telephone,may,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0


## CLEAN EDUCATION ATTRIBUTE

### Rating education type with numeric raking

In [ ]:
# Turn education level to value
conditions = [df.education == 'illiterate',
              df.education == 'basic.4y',
              df.education == 'basic.6y',
              df.education == 'basic.9y',
              df.education == 'high.school',
              df.education == 'professional.course',
              df.education == 'university.degree']

values = [1,2,3,4,5,6,7]

df.education = np.select(conditions, values, default='unknown')

In [ ]:
df.education.unique()

array(['2', '5', '3', '4', '6', 'unknown', '7', '1'], dtype=object)

In [ ]:
df.education.value_counts()

7          12168
5           9515
4           6045
6           5243
2           4176
3           2292
unknown     1731
1             18
Name: education, dtype: int64

### Filling unknown education basing their occupation (job)

We are not sure whether any diferences in education between diferent occupations. Let's check the diference by mean aggregation of education

In [ ]:
known = df[df.education != 'unknown']
known['education'] = known['education'].astype('int') #SettingWithCopyWarning

gr_known = known.groupby('job').agg(mean=('education','mean'))

gr_known

,mean
job,
admin.,6.06
blue-collar,3.54
entrepreneur,5.42
housemaid,3.69
management,6.28
retired,4.21
self-employed,5.82
services,4.82
student,5.25


In [ ]:
# This is how we get education value from each job
gr_known.loc['self-employed', 'mean'].round().astype(int).astype(str)

'6'

In [ ]:
# define function to apply fill unknown education value

def fill_unknown(row):
  if row['job'] == 'admin.':
    return gr_known.loc['admin.', 'mean'].round().astype(int).astype(str)
  
  if row['job'] == 'blue-collar':
    return gr_known.loc['blue-collar', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'entrepreneur':
    return gr_known.loc['entrepreneur', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'housemaid':
    return gr_known.loc['housemaid', 'mean'].round().astype(int).astype(str)
  
  if row['job'] == 'management':
    return gr_known.loc['management', 'mean'].round().astype(int).astype(str)
    
  if row['job'] == 'retired':
    return gr_known.loc['retired', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'self-employed':
    return gr_known.loc['self-employed', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'services':
    return gr_known.loc['services', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'student':
    return gr_known.loc['student', 'mean'].round().astype(int).astype(str)

  if row['job'] == 'technician':
    return gr_known.loc['technician', 'mean'].round().astype(int).astype(str)
  
  if row['job'] == 'unemployed':
    return gr_known.loc['unemployed', 'mean'].round().astype(int).astype(str)

  else:
    return'unknown'

In [ ]:
df.loc[df['education'] == 'unknown', 'education'] = df[df['education'] == 'unknown'].apply(fill_unknown, axis=1)

In [ ]:
df.education.value_counts()

7          12168
5           9908
4           6639
6           5856
2           4176
3           2292
unknown      131
1             18
Name: education, dtype: int64

### The rest (unknown education of unknown jobs) will be filled by mean education

In [ ]:
# The unknown level will be changed to mean value
mean = df[df.education != 'unknown'].education.astype('float32').mean()

df.education = df.education.replace('unknown', str(round(mean)))

In [ ]:
df.education.value_counts()

7    12168
5    10039
4     6639
6     5856
2     4176
3     2292
1       18
Name: education, dtype: int64

In [ ]:
# Change education column type to numeric
df.education= df.education.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   age_group       41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  int64  
 5   default         41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp.var.rate    41188 non-null  float64
 17  cons.price.idx  41188 non-null 

## PUT EVERYTHING IN 1 FUNCTION

In [ ]:
def clean_data(raw_df):
  # 1. Yes/No to numerical data
  df = raw_df.applymap(lambda x: 1 if x == 'yes' else (0 if x == 'no' else x))

  # 2. Create age group column for training data
  conditions = [(df['age'] < 25),
                (df['age'] < 35) & (df['age'] >= 25),
                (df['age'] < 60) & (df['age'] >= 35),
                (df['age'] >= 60)]
  values = [1,2,3,4]
  df['age_group'] = np.select(conditions, values)
  df = df[[df.columns[0]] + [df.columns[-1]] + list(df.columns[1:-1])] # Arrange columns order
  df.age_group = df.age_group.astype(object)

  # 3. Rating education type with numeric raking
  # 3.1. Change education object data to numerical data
  conditions = [df.education == 'illiterate',
                df.education == 'basic.4y',
                df.education == 'basic.6y',
                df.education == 'basic.9y',
                df.education == 'high.school',
                df.education == 'professional.course',
                df.education == 'university.degree']
  values = [1,2,3,4,5,6,7]
  df.education = np.select(conditions, values, default='unknown')

  # 3.2. Filling unknown education basing their occupation (job)
  # Groupby to find mean education of each job
  known = df[df.education != 'unknown']
  known['education'] = known['education'].astype('int') #SettingWithCopyWarning
  gr_known = known.groupby('job').agg(mean=('education','mean'))

  # Define function to apply fill unknown education value
  def fill_unknown(row):
    if row['job'] == 'admin.':
      return gr_known.loc['admin.', 'mean'].round().astype(int).astype(str)    
    if row['job'] == 'blue-collar':
      return gr_known.loc['blue-collar', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'entrepreneur':
      return gr_known.loc['entrepreneur', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'housemaid':
      return gr_known.loc['housemaid', 'mean'].round().astype(int).astype(str)    
    if row['job'] == 'management':
      return gr_known.loc['management', 'mean'].round().astype(int).astype(str)     
    if row['job'] == 'retired':
      return gr_known.loc['retired', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'self-employed':
      return gr_known.loc['self-employed', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'services':
      return gr_known.loc['services', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'student':
      return gr_known.loc['student', 'mean'].round().astype(int).astype(str)
    if row['job'] == 'technician':
      return gr_known.loc['technician', 'mean'].round().astype(int).astype(str)    
    if row['job'] == 'unemployed':
      return gr_known.loc['unemployed', 'mean'].round().astype(int).astype(str)
    else:
      return'unknown'
  # Apply def function
  df.loc[df['education'] == 'unknown', 'education'] = df[df['education'] == 'unknown'].apply(fill_unknown, axis=1)

  # 3.3. The rest of unknown value will be changed to mean value
  mean = df[df.education != 'unknown'].education.astype('float32').mean()
  df.education = df.education.replace('unknown', str(round(mean)))

  # 3.4. Change education column type to numeric
  df.education= df.education.astype(int)

  return df

# **EXPORT DATA**

In [ ]:
df = clean_data(raw_df)
df.head()

,age,age_group,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,housemaid,married,2,0,0,0,telephone,may,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
1,57,3,services,married,5,unknown,0,0,telephone,may,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
2,37,3,services,married,5,0,1,0,telephone,may,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
3,40,3,admin.,married,3,0,0,0,telephone,may,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0
4,56,3,services,married,5,0,0,1,telephone,may,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,0


In [ ]:
df.to_csv('bank_marketing_cleaned.csv', index=False)
!cp bank_marketing_cleaned.csv '/content/gdrive/MyDrive/Final Project'